Import libraries and upload data from nflverse github

In [1]:
#Import package
from urllib.request import urlretrieve

#Import Pandas
import pandas as pd

#Import Numpy
import numpy as np

#Import Matplotlib
import matplotlib.pyplot as plt

#Import Seaborn
import seaborn as sns

import sys
!{sys.executable} -m pip install python-dotenv requests

#Import requests
import requests

#Import datetime
from datetime import datetime


#Import beautifulsoup
from bs4 import BeautifulSoup

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


/Users/jonathan/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
#Assign Url of the file: Url
Url = "https://raw.githubusercontent.com/nflverse/nfldata/refs/heads/master/data/games.csv"

#Save the file locally
urlretrieve(Url, 'games.csv')

#Read file into a DataFrame and print head
games = pd.read_csv('games.csv')
print(games.head())

           game_id  season game_type  week     gameday weekday gametime  \
0  1999_01_MIN_ATL    1999       REG     1  1999-09-12  Sunday      NaN   
1   1999_01_KC_CHI    1999       REG     1  1999-09-12  Sunday      NaN   
2  1999_01_PIT_CLE    1999       REG     1  1999-09-12  Sunday      NaN   
3   1999_01_OAK_GB    1999       REG     1  1999-09-12  Sunday      NaN   
4  1999_01_BUF_IND    1999       REG     1  1999-09-12  Sunday      NaN   

  away_team  away_score home_team  ...  wind  away_qb_id  home_qb_id  \
0       MIN        17.0       ATL  ...   NaN  00-0003761  00-0002876   
1        KC        17.0       CHI  ...  12.0  00-0006300  00-0010560   
2       PIT        43.0       CLE  ...  12.0  00-0015700  00-0004230   
3       OAK        24.0        GB  ...  10.0  00-0005741  00-0005106   
4       BUF        14.0       IND  ...   NaN  00-0005363  00-0010346   

         away_qb_name    home_qb_name          away_coach    home_coach  \
0  Randall Cunningham  Chris Chandler    

In [3]:
#Print info of the data
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6991 entries, 0 to 6990
Data columns (total 46 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   game_id           6991 non-null   object 
 1   season            6991 non-null   int64  
 2   game_type         6991 non-null   object 
 3   week              6991 non-null   int64  
 4   gameday           6991 non-null   object 
 5   weekday           6991 non-null   object 
 6   gametime          6732 non-null   object 
 7   away_team         6991 non-null   object 
 8   away_score        6990 non-null   float64
 9   home_team         6991 non-null   object 
 10  home_score        6990 non-null   float64
 11  location          6991 non-null   object 
 12  result            6990 non-null   float64
 13  total             6990 non-null   float64
 14  overtime          6990 non-null   float64
 15  old_game_id       6991 non-null   int64  
 16  gsis              6990 non-null   float64


In [4]:
#Shape of the data
games.shape

(6991, 46)

In [5]:
#print value counts of location column to check if games are duplicated home/away
games.location.value_counts()

location
Home       6905
Neutral      86
Name: count, dtype: int64

More neutral games here than just superbowls, what are the other neutral games?

In [6]:
#Determine what Neutral games are
#Filter for neutral games and display stadium info, and game type
neutral_games = games[games.location == 'Neutral'][['game_id', 'stadium', 'game_type']]
print(neutral_games)

              game_id                  stadium game_type
258   1999_21_STL_TEN             Georgia Dome        SB
517   2000_21_BAL_NYG    Raymond James Stadium        SB
776    2001_21_STL_NE      Louisiana Superdome        SB
1043   2002_21_OAK_TB         Qualcomm Stadium        SB
1159   2003_08_MIA_SD        Sun Devil Stadium       REG
...               ...                      ...       ...
6785  2024_06_JAX_CHI        Tottenham Stadium       REG
6799   2024_07_NE_JAX          Wembley Stadium       REG
6845  2024_10_NYG_CAR            Allianz Arena       REG
6983   2024_19_MIN_LA       State Farm Stadium        WC
6990   2024_22_KC_PHI  Mercedes-Benz Superdome        SB

[86 rows x 3 columns]


It seems like most of these are games played abroad, and 2 games played to avoid wildfires, 2003 MIA @ SD regular season game and 2024 MIN @ LA wildcard game, however SD no longer has a team, I should check what teams are in this data

In [7]:
#How many teams are there?
games.home_team.value_counts()

home_team
NE     239
PHI    232
KC     230
GB     226
PIT    225
SEA    224
IND    223
DEN    221
NO     221
SF     221
TB     221
BAL    220
BUF    219
CHI    218
ATL    217
TEN    217
DAL    217
MIN    217
NYG    216
CAR    216
CIN    216
ARI    215
WAS    214
JAX    214
DET    213
MIA    213
NYJ    211
CLE    210
HOU    194
OAK    173
SD     149
STL    141
LA      80
LAC     66
LV      42
Name: count, dtype: int64

Teams have moved cities since 1999, when the data begins, OAK to LV, STL to LA, and SD to LAC, these teams should be combinded

In [8]:
#Combine teams that have moved cities for home_team and away_team
games.home_team.replace({'OAK': 'LV', 'STL': 'LA', 'SD': 'LAC'}, inplace=True)
games.away_team.replace({'OAK': 'LV', 'STL': 'LA', 'SD': 'LAC'}, inplace=True)

#check value counts of home_team and away_team
print(games.home_team.value_counts())
print(games.away_team.value_counts())

home_team
NE     239
PHI    232
KC     230
GB     226
PIT    225
SEA    224
IND    223
DEN    221
NO     221
SF     221
LA     221
TB     221
BAL    220
BUF    219
CHI    218
MIN    217
DAL    217
ATL    217
TEN    217
CIN    216
CAR    216
NYG    216
LV     215
LAC    215
ARI    215
WAS    214
JAX    214
DET    213
MIA    213
NYJ    211
CLE    210
HOU    194
Name: count, dtype: int64
away_team
BAL    232
GB     226
IND    225
SEA    225
PIT    224
NE     223
PHI    223
NYG    222
SF     222
LA     222
NYJ    221
TEN    221
MIN    220
CAR    219
KC     219
LAC    219
DAL    218
NO     218
TB     218
ATL    217
WAS    217
CIN    217
JAX    216
MIA    216
DEN    215
DET    215
ARI    215
BUF    215
CLE    214
LV     214
CHI    211
HOU    192
Name: count, dtype: int64


/var/folders/pm/v1wywnps12sf4vmw4zxrydwr0000gn/T/ipykernel_33387/2999817110.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  games.home_team.replace({'OAK': 'LV', 'STL': 'LA', 'SD': 'LAC'}, inplace=True)
/var/folders/pm/v1wywnps12sf4vmw4zxrydwr0000gn/T/ipykernel_33387/2999817110.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on wh

In [9]:
#Check for null values in each column
print(games.isnull().sum().sort_values(ascending=False))

nfl_detail_id       6719
ftn                 5615
pff                 2699
under_odds          1984
over_odds           1984
home_moneyline      1981
away_moneyline      1981
away_spread_odds    1981
home_spread_odds    1981
temp                1975
wind                1975
gametime             259
surface               43
referee                2
total                  1
away_score             1
home_score             1
result                 1
overtime               1
gsis                   1
div_game               0
roof                   0
game_id                0
home_qb_id             0
away_qb_name           0
home_qb_name           0
away_coach             0
home_coach             0
stadium_id             0
away_qb_id             0
home_rest              0
total_line             0
spread_line            0
season                 0
away_rest              0
espn                   0
pfr                    0
old_game_id            0
location               0
home_team              0


In [10]:
#Display row with missing scores
print(games[games.home_score.isnull() | games.away_score.isnull()][['game_id', 'home_team', 'away_team', 'home_score', 'away_score', 'gameday']])

             game_id home_team away_team  home_score  away_score     gameday
6990  2024_22_KC_PHI       PHI        KC         NaN         NaN  2025-02-09


The 2024 SB has not been played yet, so it is not a problem that the scores are missing.

Missing values for gambling lines can be ignored for now, as the goal for the model is to predict the outcome of the game.
They may be useful later, so I will keep them to the side.

PFF and PTN are ID numbers for premium databases, so they will be ignored in this project.

Game time is missing just for the 1999 season, this can be fixed with another dataset.

First wind and temp data can be checked.

In [ ]:
#Check for missing wind and temperature values
print("Rows with missing wind values:")
print(games[games.wind.isnull()][['game_id', 'home_team', 'away_team', 'gameday', 'wind']])
print("\nRows with missing temperature values:")
print(games[games.temp.isnull()][['game_id', 'home_team', 'away_team', 'gameday', 'temp']])

Rows with missing wind values:
              game_id home_team away_team     gameday  wind
0     1999_01_MIN_ATL       ATL       MIN  1999-09-12   NaN
4     1999_01_BUF_IND       IND       BUF  1999-09-12   NaN
6      1999_01_CAR_NO        NO       CAR  1999-09-12   NaN
9     1999_01_DET_SEA       SEA       DET  1999-09-12   NaN
10    1999_01_BAL_STL        LA       BAL  1999-09-12   NaN
...               ...       ...       ...         ...   ...
6977  2024_18_MIN_DET       DET       MIN  2025-01-05   NaN
6978  2024_19_LAC_HOU       HOU       LAC  2025-01-11   NaN
6983   2024_19_MIN_LA        LA       MIN  2025-01-13   NaN
6985  2024_20_WAS_DET       DET       WAS  2025-01-18   NaN
6990   2024_22_KC_PHI       PHI        KC  2025-02-09   NaN

[1975 rows x 5 columns]

Rows with missing temperature values:
              game_id home_team away_team     gameday  temp
0     1999_01_MIN_ATL       ATL       MIN  1999-09-12   NaN
4     1999_01_BUF_IND       IND       BUF  1999-09-12   NaN
6    

Some of these stadiums are domes, so the wind and temperature will be missing for these. If they are games played in a closed dome I will impute 0 for wind and 70 for temp, the average indoor nfl stadium temperature.

In [12]:
#Check unique values for roof
print("\nUnique values for roof:")
print(games.roof.value_counts())


Unique values for roof:
roof
outdoors    5140
dome        1144
closed       579
open         128
Name: count, dtype: int64


In [13]:
#Display rows where roof is open or closed
print("\nRows where roof is open or closed:")
print(games[games.roof.isin(['open', 'closed'])][['game_id', 'home_team', 'away_team', 'gameday', 'roof']])


Rows where roof is open or closed:
              game_id home_team away_team     gameday    roof
791   2002_01_DAL_HOU       HOU       DAL  2002-09-08  closed
810   2002_03_IND_HOU       HOU       IND  2002-09-22    open
853   2002_06_BUF_HOU       HOU       BUF  2002-10-13  closed
898   2002_09_CIN_HOU       HOU       CIN  2002-11-03  closed
923   2002_11_JAX_HOU       HOU       JAX  2002-11-17    open
...               ...       ...       ...         ...     ...
6965  2024_18_WAS_DAL       DAL       WAS  2025-01-05  closed
6967  2024_18_JAX_IND       IND       JAX  2025-01-05  closed
6972   2024_18_SF_ARI       ARI        SF  2025-01-05  closed
6978  2024_19_LAC_HOU       HOU       LAC  2025-01-11  closed
6983   2024_19_MIN_LA        LA       MIN  2025-01-13  closed

[707 rows x 5 columns]


In [14]:
# Impute wind and temp values for indoor stadiums
games.loc[(games.wind.isnull()) & (games.roof.isin(['dome', 'closed'])), 'wind'] = 0
games.loc[(games.temp.isnull()) & (games.roof.isin(['dome', 'closed'])), 'temp'] = 70
# Check remaining missing values
print("\nRemaining rows with missing wind values:")
print(games[games.wind.isnull()][['game_id', 'home_team', 'away_team', 'gameday', 'wind', 'roof']])
print("\nRemaining rows with missing temperature values:") 
print(games[games.temp.isnull()][['game_id', 'home_team', 'away_team', 'gameday', 'temp', 'roof']])


Remaining rows with missing wind values:
              game_id home_team away_team     gameday  wind      roof
334   2000_06_BUF_MIA       MIA       BUF  2000-10-08   NaN  outdoors
467    2000_15_CAR_KC        KC       CAR  2000-12-10   NaN  outdoors
810   2002_03_IND_HOU       HOU       IND  2002-09-22   NaN      open
923   2002_11_JAX_HOU       HOU       JAX  2002-11-17   NaN      open
949   2002_12_NYG_HOU       HOU       NYG  2002-11-24   NaN      open
...               ...       ...       ...         ...   ...       ...
6707   2024_01_GB_PHI       PHI        GB  2024-09-06   NaN  outdoors
6845  2024_10_NYG_CAR       CAR       NYG  2024-11-10   NaN  outdoors
6891  2024_13_HOU_JAX       JAX       HOU  2024-12-01   NaN  outdoors
6918  2024_15_NYJ_JAX       JAX       NYJ  2024-12-15   NaN  outdoors
6961   2024_17_DET_SF        SF       DET  2024-12-30   NaN  outdoors

[252 rows x 6 columns]

Remaining rows with missing temperature values:
              game_id home_team away_team    

Ill use averages for the remaining missing wind and temp since its only about 250 games out of about 7000

In [15]:
#Games with outdoor or open stadium in roof column
games_filtered = games[games['roof'].isin(["open", "outdoors"])]

#Calculate mean of 'wind' and 'temp' columns from filtered rows
mean_wind = games_filtered['wind'].mean()
mean_temp = games_filtered['wind'].mean()

#Impute missing values from calculated average
games['wind'].fillna(mean_wind, inplace=True)
games['temp'].fillna(mean_temp, inplace=True)

# Check remaining missing values
print("\nRemaining rows with missing wind values:")
print(games[games.wind.isnull()][['game_id', 'home_team', 'away_team', 'gameday', 'wind', 'roof']])
print("\nRemaining rows with missing temperature values:") 
print(games[games.temp.isnull()][['game_id', 'home_team', 'away_team', 'gameday', 'temp', 'roof']])


Remaining rows with missing wind values:
Empty DataFrame
Columns: [game_id, home_team, away_team, gameday, wind, roof]
Index: []

Remaining rows with missing temperature values:
Empty DataFrame
Columns: [game_id, home_team, away_team, gameday, temp, roof]
Index: []


/var/folders/pm/v1wywnps12sf4vmw4zxrydwr0000gn/T/ipykernel_33387/3996895605.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  games['wind'].fillna(mean_wind, inplace=True)
/var/folders/pm/v1wywnps12sf4vmw4zxrydwr0000gn/T/ipykernel_33387/3996895605.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always

In [16]:
# Check for missing game times
print("Rows with missing game times:")
print(games[games.gametime.isnull()][['game_id', 'home_team', 'away_team', 'gameday', 'gametime']])

Rows with missing game times:
             game_id home_team away_team     gameday gametime
0    1999_01_MIN_ATL       ATL       MIN  1999-09-12      NaN
1     1999_01_KC_CHI       CHI        KC  1999-09-12      NaN
2    1999_01_PIT_CLE       CLE       PIT  1999-09-12      NaN
3     1999_01_OAK_GB        GB        LV  1999-09-12      NaN
4    1999_01_BUF_IND       IND       BUF  1999-09-12      NaN
..               ...       ...       ...         ...      ...
254  1999_19_TEN_IND       IND       TEN  2000-01-16      NaN
255  1999_19_MIN_STL        LA       MIN  2000-01-16      NaN
256  1999_20_TEN_JAX       JAX       TEN  2000-01-23      NaN
257   1999_20_TB_STL        LA        TB  2000-01-23      NaN
258  1999_21_STL_TEN       TEN        LA  2000-01-30      NaN

[259 rows x 5 columns]


To get times I will be bringing in another dataframe from pro football reference and taking times from there.

In [17]:
url2 = 'https://www.pro-football-reference.com/years/1999/games.htm#games'
# Send GET request and create BeautifulSoup object
response = requests.get(url2)
soup = BeautifulSoup(response.content, 'html.parser')

# Find the main games table
table = soup.find('table', id='games')

# Initialize lists to store data
data = []

# Extract data from table rows
for row in table.find_all('tr')[1:]:  # Skip header row
    cols = row.find_all(['th', 'td'])
    if len(cols) > 0:
        week = cols[0].text.strip()
        day = cols[1].text.strip()
        date = cols[2].text.strip()
        time = cols[3].text.strip()
        winner = cols[4].text.strip()
        loser = cols[6].text.strip()
        pts_w = cols[7].text.strip()
        pts_l = cols[8].text.strip()
        yds_w = cols[9].text.strip()
        to_w = cols[10].text.strip()
        yds_l = cols[11].text.strip()
        to_l = cols[12].text.strip()
        
        data.append([week, day, date, time, winner, loser, pts_w, pts_l, yds_w, to_w, yds_l, to_l])

# Create DataFrame
games1999 = pd.DataFrame(data, columns=['Week', 'Day', 'Date', 'Time', 'Winner', 'Loser', 
                                      'PtsW', 'PtsL', 'YdsW', 'TOW', 'YdsL', 'TOL'])
 #print games1999 head
print(games1999.head())
print(games1999.info())
print(games1999.shape)

  Week  Day        Date    Time              Winner                Loser  \
0    1  Sun  1999-09-12  1:00PM      Dallas Cowboys  Washington Redskins   
1    1  Sun  1999-09-12  1:02PM  Indianapolis Colts        Buffalo Bills   
2    1  Sun  1999-09-12  1:02PM  New Orleans Saints    Carolina Panthers   
3    1  Sun  1999-09-12  1:02PM       Chicago Bears   Kansas City Chiefs   
4    1  Sun  1999-09-12  1:02PM   Green Bay Packers      Oakland Raiders   

       PtsW PtsL YdsW  TOW YdsL  TOL  
0  boxscore   41   35  541    3  504  
1  boxscore   31   14  393    3  321  
2  boxscore   19   10  255    0  246  
3  boxscore   20   17  388    2  349  
4  boxscore   28   24  409    4  363  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276 entries, 0 to 275
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Week    276 non-null    object
 1   Day     276 non-null    object
 2   Date    276 non-null    object
 3   Time    276 non-nul

In [18]:
# Create dictionary for team name to abbreviation mapping
team_mapping = {
    'Arizona Cardinals': 'ARI', 'Atlanta Falcons': 'ATL', 'Baltimore Ravens': 'BAL', 
    'Buffalo Bills': 'BUF', 'Carolina Panthers': 'CAR', 'Chicago Bears': 'CHI',
    'Cincinnati Bengals': 'CIN', 'Cleveland Browns': 'CLE', 'Dallas Cowboys': 'DAL',
    'Denver Broncos': 'DEN', 'Detroit Lions': 'DET', 'Green Bay Packers': 'GB',
    'Houston Texans': 'HOU', 'Indianapolis Colts': 'IND', 'Jacksonville Jaguars': 'JAX',
    'Kansas City Chiefs': 'KC', 'Los Angeles Chargers': 'LAC', 'Los Angeles Rams': 'LA',
    'Las Vegas Raiders': 'LV', 'Miami Dolphins': 'MIA', 'Minnesota Vikings': 'MIN',
    'New England Patriots': 'NE', 'New Orleans Saints': 'NO', 'New York Giants': 'NYG',
    'New York Jets': 'NYJ', 'Philadelphia Eagles': 'PHI', 'Pittsburgh Steelers': 'PIT',
    'San Francisco 49ers': 'SF', 'Seattle Seahawks': 'SEA', 'Tampa Bay Buccaneers': 'TB',
    'Tennessee Titans': 'TEN', 'Washington Commanders': 'WAS',
    # Add historical team names
    'Oakland Raiders': 'LV', 'San Diego Chargers': 'LAC', 'St. Louis Rams': 'LA',
    'Washington Redskins': 'WAS', 'Washington Football Team': 'WAS'
}

# Replace team names with abbreviations
games1999['Winner'] = games1999['Winner'].replace(team_mapping)
games1999['Loser'] = games1999['Loser'].replace(team_mapping)

# Verify the changes
print("\nAfter team name conversion:")
print(games1999[['Winner', 'Loser']].head())


After team name conversion:
  Winner Loser
0    DAL   WAS
1    IND   BUF
2     NO   CAR
3    CHI    KC
4     GB    LV


In [19]:
#Check unique values of Time
print("Unique time values:")
print(games1999['Time'].unique())

# Define a function to convert time to 24-hour format
def convert_to_24hr(time_str):
    try:
        # Remove any extra spaces
        time_str = time_str.strip()
        # Convert to datetime using pandas
        time_obj = pd.to_datetime(time_str).strftime('%H:%M')
        return time_obj
    except:
        return time_str  # Return original if conversion fails

# Apply the conversion
games1999['Time'] = games1999['Time'].apply(convert_to_24hr)

# Check the results
print("\nAfter conversion:")
print(games1999['Time'].head())

Unique time values:
['1:00PM' '1:02PM' '1:03PM' '1:04PM' '4:15PM' '4:16PM' '8:26PM' '9:07PM'
 'Time' '1:01PM' '4:05PM' '8:22PM' '8:23PM' '8:08PM' '1:05PM' '4:18PM'
 '8:24PM' '4:06PM' '4:17PM' '9:09PM' '8:21PM' '1:06PM' '5:15PM' '2:00PM'
 '2:55PM' '8:20PM' '9:10PM' '12:00PM' '12:03PM' '9:08PM' '5:05PM'
 '11:07PM' '12:35PM' '1:10PM' '4:07PM' '1:15PM' '9:05PM' '2:05PM'
 '12:41PM' '2:15PM' '6:23PM' '9:15PM' '3:00PM' '' '12:32PM' '12:42PM'
 '4:22PM' '6:25PM']

After conversion:
0    13:00
1    13:02
2    13:02
3    13:02
4    13:02
Name: Time, dtype: object


In [20]:
# Get unique values from gametime column
print("Unique game times:")
print(games['gametime'].unique())

Unique game times:
[nan '13:00' '16:05' '16:15' '20:20' '09:00' '12:00' '15:05' '15:15'
 '13:05' '12:30' '16:00' '18:18' '20:30' '13:30' '17:00' '16:30' '20:00'
 '18:40' '15:00' '18:30' '20:15' '14:00' '16:45' '18:45' '18:25' '19:30'
 '19:00' '22:15' '16:25' '16:35' '20:25' '19:10' '22:25' '20:40' '18:31'
 '16:40' '18:32' '21:15' '19:20' '23:30' '20:10' '21:20' '09:30' '18:50'
 '22:20' '21:30' '16:36' '18:41' '19:15' '22:10' '19:05' '21:00' '15:40'
 '20:05' '16:50']


In [21]:
print(games[games['gametime'].isnull()])

             game_id  season game_type  week     gameday weekday gametime  \
0    1999_01_MIN_ATL    1999       REG     1  1999-09-12  Sunday      NaN   
1     1999_01_KC_CHI    1999       REG     1  1999-09-12  Sunday      NaN   
2    1999_01_PIT_CLE    1999       REG     1  1999-09-12  Sunday      NaN   
3     1999_01_OAK_GB    1999       REG     1  1999-09-12  Sunday      NaN   
4    1999_01_BUF_IND    1999       REG     1  1999-09-12  Sunday      NaN   
..               ...     ...       ...   ...         ...     ...      ...   
254  1999_19_TEN_IND    1999       DIV    19  2000-01-16  Sunday      NaN   
255  1999_19_MIN_STL    1999       DIV    19  2000-01-16  Sunday      NaN   
256  1999_20_TEN_JAX    1999       CON    20  2000-01-23  Sunday      NaN   
257   1999_20_TB_STL    1999       CON    20  2000-01-23  Sunday      NaN   
258  1999_21_STL_TEN    1999        SB    21  2000-01-30  Sunday      NaN   

    away_team  away_score home_team  ...  wind  away_qb_id  home_qb_id  \
0

We will add a winner column now

In [22]:
# Create Winner column based on scores and teams
games['Winner'] = games.apply(lambda row: 
    row['home_team'] if row['home_score'] > row['away_score']
    else row['away_team'] if row['away_score'] > row['home_score']
    else 'Tie', axis=1)

# Verify the results
print("\nSample of Winners:")
print(games[['home_team', 'away_team', 'home_score', 'away_score', 'Winner']].head())


Sample of Winners:
  home_team away_team  home_score  away_score Winner
0       ATL       MIN        14.0        17.0    MIN
1       CHI        KC        20.0        17.0    CHI
2       CLE       PIT         0.0        43.0    PIT
3        GB        LV        28.0        24.0     GB
4       IND       BUF        31.0        14.0    IND


In [23]:
# Create a mapping of game details to times from games1999
time_mapping = games1999.set_index(['Winner', 'Date'])['Time'].to_dict()

# Function to look up time based on winner and date
def get_time(row):
    if pd.isna(row['gametime']):
        try:
            # Try to find matching game in 1999 data
            return time_mapping.get((row['Winner'], row['gameday']))
        except:
            return row['gametime']
    return row['gametime']

# Fill missing times
games['gametime'] = games.apply(get_time, axis=1)

# Check results
print("\nMissing times after filling:")
print(games['gametime'].isna().sum())
print("\nSample of filled times:")
print(games[['gameday', 'Winner', 'gametime']].head())


Missing times after filling:
0

Sample of filled times:
      gameday Winner gametime
0  1999-09-12    MIN    16:15
1  1999-09-12    CHI    13:02
2  1999-09-12    PIT    20:26
3  1999-09-12     GB    13:02
4  1999-09-12    IND    13:02


Times are now filled in, wind and temp is done.

In [24]:
print(games.isnull().sum().sort_values(ascending=False))

nfl_detail_id       6719
ftn                 5615
pff                 2699
under_odds          1984
over_odds           1984
home_moneyline      1981
away_moneyline      1981
away_spread_odds    1981
home_spread_odds    1981
surface               43
referee                2
away_score             1
result                 1
total                  1
overtime               1
gsis                   1
home_score             1
div_game               0
roof                   0
temp                   0
game_id                0
wind                   0
home_qb_id             0
away_qb_name           0
home_qb_name           0
away_coach             0
home_coach             0
stadium_id             0
stadium                0
away_qb_id             0
home_rest              0
total_line             0
spread_line            0
season                 0
away_rest              0
espn                   0
pfr                    0
old_game_id            0
location               0
home_team              0


Now we will drop columns we know we won't be using for this project, and adding in data for yards and turnovers from each game.

In [25]:
#List of columns to drop
todrop = ['gsis', 'nfl_detail_id', 'pfr', 'pff', 'espn', 'ftn', 'surface', 'away_qb_id',	'home_qb_id', 'away_qb_name', 'home_qb_name', 'away_coach', 'home_coach', 'referee']

#Drop columns
games.drop(todrop, axis=1, inplace=True)

In [26]:
#Check new datframe info
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6991 entries, 0 to 6990
Data columns (total 33 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   game_id           6991 non-null   object 
 1   season            6991 non-null   int64  
 2   game_type         6991 non-null   object 
 3   week              6991 non-null   int64  
 4   gameday           6991 non-null   object 
 5   weekday           6991 non-null   object 
 6   gametime          6991 non-null   object 
 7   away_team         6991 non-null   object 
 8   away_score        6990 non-null   float64
 9   home_team         6991 non-null   object 
 10  home_score        6990 non-null   float64
 11  location          6991 non-null   object 
 12  result            6990 non-null   float64
 13  total             6990 non-null   float64
 14  overtime          6990 non-null   float64
 15  old_game_id       6991 non-null   int64  
 16  away_rest         6991 non-null   int64  


In [27]:
games.head()

,game_id,season,game_type,week,gameday,weekday,gametime,away_team,away_score,home_team,...,total_line,under_odds,over_odds,div_game,roof,temp,wind,stadium_id,stadium,Winner
0,1999_01_MIN_ATL,1999,REG,1,1999-09-12,Sunday,16:15,MIN,17.0,ATL,...,49.0,NaN,NaN,0,dome,70.0,0.0,ATL00,Georgia Dome,MIN
1,1999_01_KC_CHI,1999,REG,1,1999-09-12,Sunday,13:02,KC,17.0,CHI,...,38.0,NaN,NaN,0,outdoors,80.0,12.0,CHI98,Soldier Field,CHI
2,1999_01_PIT_CLE,1999,REG,1,1999-09-12,Sunday,20:26,PIT,43.0,CLE,...,37.0,NaN,NaN,1,outdoors,78.0,12.0,CLE00,Cleveland Browns Stadium,PIT
3,1999_01_OAK_GB,1999,REG,1,1999-09-12,Sunday,13:02,LV,24.0,GB,...,43.0,NaN,NaN,0,outdoors,67.0,10.0,GNB00,Lambeau Field,GB
4,1999_01_BUF_IND,1999,REG,1,1999-09-12,Sunday,13:02,BUF,14.0,IND,...,45.5,NaN,NaN,1,dome,70.0,0.0,IND99,RCA Dome,IND


In [28]:
#bring in dataframe for 2023 season play by play
df23 = pd.read_csv('pbp-2023.csv')
df23.head()

,GameId,GameDate,Quarter,Minute,Second,OffenseTeam,DefenseTeam,Down,ToGo,YardLine,...,IsTwoPointConversion,IsTwoPointConversionSuccessful,RushDirection,YardLineFixed,YardLineDirection,IsPenaltyAccepted,PenaltyTeam,IsNoPlay,PenaltyType,PenaltyYards
0,2023110503,2023-11-05,2,12,5,CLE,ARI,3,5,56,...,0,0,NaN,44,OPP,0,NaN,0,NaN,0
1,2023110504,2023-11-05,2,3,53,LA,GB,1,10,55,...,0,0,NaN,45,OPP,0,NaN,0,NaN,0
2,2023110504,2023-11-05,2,3,15,LA,GB,2,2,63,...,0,0,CENTER,37,OPP,0,NaN,0,NaN,0
3,2023110504,2023-11-05,2,0,5,GB,LA,2,4,70,...,0,0,NaN,30,OPP,0,NaN,0,NaN,0
4,2023110504,2023-11-05,2,0,4,GB,LA,3,4,70,...,0,0,NaN,30,OPP,0,NaN,0,NaN,0


In [29]:
#Info of 2023 season dataframe
df23.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51287 entries, 0 to 51286
Data columns (total 45 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   GameId                          51287 non-null  int64  
 1   GameDate                        51287 non-null  object 
 2   Quarter                         51287 non-null  int64  
 3   Minute                          51287 non-null  int64  
 4   Second                          51287 non-null  int64  
 5   OffenseTeam                     51287 non-null  object 
 6   DefenseTeam                     51287 non-null  object 
 7   Down                            51287 non-null  int64  
 8   ToGo                            51287 non-null  int64  
 9   YardLine                        51287 non-null  int64  
 10  Unnamed: 10                     0 non-null      float64
 11  SeriesFirstDown                 51287 non-null  int64  
 12  Unnamed: 12                     

In [30]:
#df23 null counts
print(df23.isnull().sum().sort_values(ascending=False))

Unnamed: 12                       51287
Challenger                        51287
Unnamed: 10                       51287
Unnamed: 17                       51287
Unnamed: 16                       51287
PenaltyType                       47857
PenaltyTeam                       47857
RushDirection                     38068
PassType                          32115
PlayType                           5485
Formation                           830
IsInterception                        0
IsChallenge                           0
IsChallengeReversed                   0
IsMeasurement                         0
GameId                                0
IsFumble                              0
IsPenalty                             0
IsTwoPointConversion                  0
IsTwoPointConversionSuccessful        0
YardLineFixed                         0
YardLineDirection                     0
IsPenaltyAccepted                     0
IsNoPlay                              0
IsSack                                0


In [31]:
#Create dataframe of each game with desired features
#List of game ID
df23["GameId"].unique()

array([2023110503, 2023110504, 2023110505, 2023110508, 2023110506,
       2023110507, 2023110509, 2023110510, 2023110600, 2023110900,
       2023111201, 2023111207, 2023111202, 2023111203, 2023111206,
       2023111205, 2023111211, 2023111600, 2023111904, 2023111903,
       2023111909, 2023111905, 2023111908, 2023111906, 2023111910,
       2023112300, 2023111911, 2023112302, 2023112000, 2023112301,
       2023112600, 2023112603, 2023112601, 2023112602, 2023112605,
       2023112604, 2023112606, 2023112607, 2023112609, 2023112610,
       2023112700, 2023113000, 2023120308, 2023120310, 2023121000,
       2023121002, 2023120400, 2023120700, 2023121001, 2023121003,
       2023121004, 2023121008, 2023121011, 2023121005, 2023121010,
       2023121007, 2023121101, 2023121400, 2023121009, 2023121602,
       2023121100, 2023121705, 2023121601, 2023121700, 2023121701,
       2023121600, 2023121702, 2023121703, 2023110501, 2023121704,
       2024010710, 2024010713, 2024010709, 2024010712, 2024010

In [32]:
# List of URLs you want to scrape
urls = [
    'https://www.pro-football-reference.com/years/2024/games.htm',
    'https://www.pro-football-reference.com/years/2023/games.htm',
    'https://www.pro-football-reference.com/years/2022/games.htm',
    'https://www.pro-football-reference.com/years/2021/games.htm',
    'https://www.pro-football-reference.com/years/2020/games.htm',
    'https://www.pro-football-reference.com/years/2019/games.htm',
    'https://www.pro-football-reference.com/years/2018/games.htm',
    'https://www.pro-football-reference.com/years/2017/games.htm'
]

# Function to scrape data from a single URL and return a DataFrame
def scrape_games_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the main games table
    table = soup.find('table', id='games')

    # Initialize a list to store data
    data = []

    # Extract data from table rows
    for row in table.find_all('tr')[1:]:  # Skip header row
        cols = row.find_all(['th', 'td'])
        if len(cols) > 0:
            week = cols[0].text.strip()
            day = cols[1].text.strip()
            date = cols[2].text.strip()
            time = cols[3].text.strip()
            winner = cols[4].text.strip()
            awaygame = cols[5].text.strip()
            loser = cols[6].text.strip()
            pts_w = cols[8].text.strip()
            pts_l = cols[9].text.strip()
            yds_w = cols[10].text.strip()
            to_w = cols[11].text.strip()
            yds_l = cols[12].text.strip()
            to_l = cols[13].text.strip()

            data.append([week, day, date, time, winner, awaygame, loser, pts_w, pts_l, yds_w, to_w, yds_l, to_l])

    # Create DataFrame
    games_df = pd.DataFrame(data, columns=['Week', 'Day', 'Date', 'Time', 'Winner', 'Awaygame', 'Loser', 
                                           'PtsW', 'PtsL', 'YdsW', 'TOW', 'YdsL', 'TOL'])
    return games_df

# List to store DataFrames
all_games_data = []

# Loop over the URLs and scrape data
for url in urls:
    df = scrape_games_data(url)
    all_games_data.append(df)

# Optionally, concatenate all the DataFrames into one large DataFrame
combined_games_data = pd.concat(all_games_data, ignore_index=True)

# Print the first few rows of the combined DataFrame
print(combined_games_data.head())
print(combined_games_data.info())
print(combined_games_data.shape)

  Week  Day        Date    Time               Winner Awaygame  \
0    1  Thu  2024-09-05  8:20PM   Kansas City Chiefs            
1    1  Fri  2024-09-06  8:15PM  Philadelphia Eagles            
2    1  Sun  2024-09-08  1:00PM  Pittsburgh Steelers        @   
3    1  Sun  2024-09-08  1:00PM        Buffalo Bills            
4    1  Sun  2024-09-08  1:00PM   New Orleans Saints            

               Loser PtsW PtsL YdsW TOW YdsL TOL  
0   Baltimore Ravens   27   20  353   1  452   1  
1  Green Bay Packers   34   29  410   3  414   1  
2    Atlanta Falcons   18   10  270   0  226   3  
3  Arizona Cardinals   34   28  352   1  270   1  
4  Carolina Panthers   47   10  379   1  193   3  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2349 entries, 0 to 2348
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Week      2349 non-null   object
 1   Day       2349 non-null   object
 2   Date      2349 non-null   object
 3   T

In [33]:
#Data describe
print(combined_games_data.describe())

        Week   Day  Date    Time      Winner Awaygame      Loser  PtsW  PtsL  \
count   2349  2349  2349    2349        2349     2349       2349  2349  2349   
unique    24     9   478      34          37        3         37    56    45   
top     Week   Sun  Date  1:00PM  Winner/tie           Loser/tie    27    17   
freq     132  1821   132    1099         132     1346        132   182   222   

        YdsW   TOW  YdsL   TOL  
count   2349  2349  2349  2349  
unique   377     9   404    10  
top     YdsW     0  YdsL     1  
freq     132   920   132   687  


In [34]:
#Check awaygame column
print(combined_games_data["Awaygame"].unique())

['' '@' 'N']


In [35]:
#Check if ties are indicated correctly with known tie games, will use Detroit Lions vs Pittsburgh Steelers in week 10 of 2021 season
combined_games_data[combined_games_data["Date"].isin(["2021-11-14"])]

,Week,Day,Date,Time,Winner,Awaygame,Loser,PtsW,PtsL,YdsW,TOW,YdsL,TOL
1054,10,Sun,2021-11-14,1:00PM,Dallas Cowboys,,Atlanta Falcons,43,3,431,1,214,3
1055,10,Sun,2021-11-14,1:00PM,Buffalo Bills,@,New York Jets,45,17,489,2,366,5
1056,10,Sun,2021-11-14,1:00PM,Pittsburgh Steelers,,Detroit Lions,16,16,387,3,306,0
1057,10,Sun,2021-11-14,1:00PM,Tennessee Titans,,New Orleans Saints,23,21,264,0,373,1
1058,10,Sun,2021-11-14,1:00PM,Washington Football Team,,Tampa Bay Buccaneers,29,19,320,1,273,2
1059,10,Sun,2021-11-14,1:00PM,New England Patriots,,Cleveland Browns,45,7,452,0,217,1
1060,10,Sun,2021-11-14,1:00PM,Indianapolis Colts,,Jacksonville Jaguars,23,17,295,0,331,1
1061,10,Sun,2021-11-14,4:05PM,Minnesota Vikings,@,Los Angeles Chargers,27,20,381,1,253,1
1062,10,Sun,2021-11-14,4:05PM,Carolina Panthers,@,Arizona Cardinals,34,10,341,2,169,2
1063,10,Sun,2021-11-14,4:25PM,Philadelphia Eagles,@,Denver Broncos,30,13,386,1,308,1


In [36]:
#Adding tie column
combined_games_data['Tie'] = (combined_games_data['PtsW'] == combined_games_data['PtsL']).astype(int)

In [37]:
#Look at tie games
combined_games_data[combined_games_data["Tie"]==1]

,Week,Day,Date,Time,Winner,Awaygame,Loser,PtsW,PtsL,YdsW,TOW,YdsL,TOL,Tie
289,,,Playoffs,,,,,,,,,,,1
302,SuperBowl,Sun,2025-02-09,6:30PM,Kansas City Chiefs,@,Philadelphia Eagles,,,,,,,1
592,,,Playoffs,,,,,,,,,,,1
611,1,Sun,2022-09-11,1:00PM,Houston Texans,,Indianapolis Colts,20,20,299,1,517,2,1
803,13,Sun,2022-12-04,1:00PM,Washington Commanders,@,New York Giants,20,20,411,1,316,1,1
894,,,Playoffs,,,,,,,,,,,1
1056,10,Sun,2021-11-14,1:00PM,Pittsburgh Steelers,,Detroit Lions,16,16,387,3,306,0,1
1197,,,Playoffs,,,,,,,,,,,1
1248,3,Sun,2020-09-27,1:00PM,Philadelphia Eagles,,Cincinnati Bengals,23,23,378,2,304,0,1
1483,,,Playoffs,,,,,,,,,,,1


In [38]:
#Look for non-dates in date column
print(combined_games_data["Date"].unique())

['2024-09-05' '2024-09-06' '2024-09-08' '2024-09-09' 'Date' '2024-09-12'
 '2024-09-15' '2024-09-16' '2024-09-19' '2024-09-22' '2024-09-23'
 '2024-09-26' '2024-09-29' '2024-09-30' '2024-10-03' '2024-10-06'
 '2024-10-07' '2024-10-10' '2024-10-13' '2024-10-14' '2024-10-17'
 '2024-10-20' '2024-10-21' '2024-10-24' '2024-10-27' '2024-10-28'
 '2024-10-31' '2024-11-03' '2024-11-04' '2024-11-07' '2024-11-10'
 '2024-11-11' '2024-11-14' '2024-11-17' '2024-11-18' '2024-11-21'
 '2024-11-24' '2024-11-25' '2024-11-28' '2024-11-29' '2024-12-01'
 '2024-12-02' '2024-12-05' '2024-12-08' '2024-12-09' '2024-12-12'
 '2024-12-15' '2024-12-16' '2024-12-19' '2024-12-21' '2024-12-22'
 '2024-12-23' '2024-12-25' '2024-12-26' '2024-12-28' '2024-12-29'
 '2024-12-30' '2025-01-04' '2025-01-05' 'Playoffs' '2025-01-11'
 '2025-01-12' '2025-01-13' '2025-01-18' '2025-01-19' '2025-01-26'
 '2025-02-09' '2023-09-07' '2023-09-10' '2023-09-11' '2023-09-14'
 '2023-09-17' '2023-09-18' '2023-09-21' '2023-09-24' '2023-09-25'
 '202

In [39]:
#Look at rows containing non dates in date column
searchvalues = ["Date", "Playoffs"]
print(combined_games_data[combined_games_data["Date"].isin(searchvalues)])

      Week  Day      Date  Time      Winner Awaygame      Loser  PtsW  PtsL  \
16    Week  Day      Date  Time  Winner/tie           Loser/tie  PtsW  PtsL   
33    Week  Day      Date  Time  Winner/tie           Loser/tie  PtsW  PtsL   
50    Week  Day      Date  Time  Winner/tie           Loser/tie  PtsW  PtsL   
67    Week  Day      Date  Time  Winner/tie           Loser/tie  PtsW  PtsL   
82    Week  Day      Date  Time  Winner/tie           Loser/tie  PtsW  PtsL   
...    ...  ...       ...   ...         ...      ...        ...   ...   ...   
2269  Week  Day      Date  Time  Winner/tie           Loser/tie  PtsW  PtsL   
2286  Week  Day      Date  Time  Winner/tie           Loser/tie  PtsW  PtsL   
2303  Week  Day      Date  Time  Winner/tie           Loser/tie  PtsW  PtsL   
2320  Week  Day      Date  Time  Winner/tie           Loser/tie  PtsW  PtsL   
2337             Playoffs                                                     

      YdsW  TOW  YdsL  TOL  Tie  
16    YdsW  TOW  

In [40]:
# Convert 'Date' column to datetime, with coercion of errors (invalid dates become NaT)
combined_games_data["Date"] = pd.to_datetime(combined_games_data["Date"], errors='coerce')

# Drop rows with NaT (invalid dates) if needed
combined_games_data = combined_games_data.dropna(subset=["Date"])

# Set 'Date' column as index
combined_games_data.set_index("Date", inplace=True)

# Print the DataFrame
print(combined_games_data)

                 Week  Day    Time                Winner Awaygame  \
Date                                                                
2024-09-05          1  Thu  8:20PM    Kansas City Chiefs            
2024-09-06          1  Fri  8:15PM   Philadelphia Eagles            
2024-09-08          1  Sun  1:00PM   Pittsburgh Steelers        @   
2024-09-08          1  Sun  1:00PM         Buffalo Bills            
2024-09-08          1  Sun  1:00PM    New Orleans Saints            
...               ...  ...     ...                   ...      ...   
2018-01-14   Division  Sun  1:05PM  Jacksonville Jaguars        @   
2018-01-14   Division  Sun  4:40PM     Minnesota Vikings            
2018-01-21  ConfChamp  Sun  3:05PM  New England Patriots            
2018-01-21  ConfChamp  Sun  6:40PM   Philadelphia Eagles            
2018-02-04  SuperBowl  Sun  6:30PM   Philadelphia Eagles        N   

                           Loser PtsW PtsL YdsW TOW YdsL TOL  Tie  
Date                              

In [41]:
#Look at tie games again
combined_games_data[combined_games_data["Tie"]==1]

,Week,Day,Time,Winner,Awaygame,Loser,PtsW,PtsL,YdsW,TOW,YdsL,TOL,Tie
Date,,,,,,,,,,,,,
2025-02-09,SuperBowl,Sun,6:30PM,Kansas City Chiefs,@,Philadelphia Eagles,,,,,,,1
2022-09-11,1,Sun,1:00PM,Houston Texans,,Indianapolis Colts,20,20,299,1,517,2,1
2022-12-04,13,Sun,1:00PM,Washington Commanders,@,New York Giants,20,20,411,1,316,1,1
2021-11-14,10,Sun,1:00PM,Pittsburgh Steelers,,Detroit Lions,16,16,387,3,306,0,1
2020-09-27,3,Sun,1:00PM,Philadelphia Eagles,,Cincinnati Bengals,23,23,378,2,304,0,1
2019-09-08,1,Sun,4:25PM,Detroit Lions,@,Arizona Cardinals,27,27,477,2,387,1,1
2018-09-09,1,Sun,1:00PM,Pittsburgh Steelers,@,Cleveland Browns,21,21,472,6,327,1,1
2018-09-16,2,Sun,1:00PM,Minnesota Vikings,@,Green Bay Packers,29,29,480,1,351,0,1


Superbowl data will be available after this weekend, should solve the tie value there

In [42]:
#Check for missing values
print(combined_games_data.isnull().sum().sort_values(ascending=False))

Week        0
Day         0
Time        0
Winner      0
Awaygame    0
Loser       0
PtsW        0
PtsL        0
YdsW        0
TOW         0
YdsL        0
TOL         0
Tie         0
dtype: int64


In [43]:
combined_games_data.dtypes

Week        object
Day         object
Time        object
Winner      object
Awaygame    object
Loser       object
PtsW        object
PtsL        object
YdsW        object
TOW         object
YdsL        object
TOL         object
Tie          int64
dtype: object

In [44]:
#converting numeric data
combined_games_data['PtsW'] = pd.to_numeric(combined_games_data['PtsW'], errors='coerce')
combined_games_data['PtsL'] = pd.to_numeric(combined_games_data['PtsL'], errors='coerce')
combined_games_data['YdsW'] = pd.to_numeric(combined_games_data['YdsW'], errors='coerce')
combined_games_data['YdsL'] = pd.to_numeric(combined_games_data['YdsL'], errors='coerce')
combined_games_data['TOW'] = pd.to_numeric(combined_games_data['TOW'], errors='coerce')
combined_games_data['TOL'] = pd.to_numeric(combined_games_data['TOL'], errors='coerce')

In [45]:
#Rename df
NFL = combined_games_data

In [46]:
#Feature engineering for differentials (yards, turnovers, score)
#Points differential
NFL["PointDiff"] = NFL["PtsW"] - NFL ["PtsL"]

#Yard differential
NFL["YardDiff"] = NFL["YdsW"] - NFL["YdsL"]

#Turnover differential
NFL["TurnoverDiff"] = NFL["TOW"] - NFL["TOL"]

In [47]:
print(NFL.head())

           Week  Day    Time               Winner Awaygame              Loser  \
Date                                                                            
2024-09-05    1  Thu  8:20PM   Kansas City Chiefs            Baltimore Ravens   
2024-09-06    1  Fri  8:15PM  Philadelphia Eagles           Green Bay Packers   
2024-09-08    1  Sun  1:00PM  Pittsburgh Steelers        @    Atlanta Falcons   
2024-09-08    1  Sun  1:00PM        Buffalo Bills           Arizona Cardinals   
2024-09-08    1  Sun  1:00PM   New Orleans Saints           Carolina Panthers   

            PtsW  PtsL   YdsW  TOW   YdsL  TOL  Tie  PointDiff  YardDiff  \
Date                                                                       
2024-09-05  27.0  20.0  353.0  1.0  452.0  1.0    0        7.0     -99.0   
2024-09-06  34.0  29.0  410.0  3.0  414.0  1.0    0        5.0      -4.0   
2024-09-08  18.0  10.0  270.0  0.0  226.0  3.0    0        8.0      44.0   
2024-09-08  34.0  28.0  352.0  1.0  270.0  1.0    0 

In [48]:
# Create the 'Home' and 'Away' columns based on the 'Awaygame' column
NFL['Home'] = NFL['Awaygame'].apply(lambda x: True if x == '' else False)
NFL['Away'] = NFL['Awaygame'].apply(lambda x: True if x == '@' else False)

# For neutral games (N), we can set both as False (or leave them as NaN if you prefer)
NFL.loc[NFL['Awaygame'] == 'N', ['Home', 'Away']] = False, False

# Check the new columns
NFL[['Awaygame', 'Home', 'Away']].head()

,Awaygame,Home,Away
Date,,,
2024-09-05,,True,False
2024-09-06,,True,False
2024-09-08,@,False,True
2024-09-08,,True,False
2024-09-08,,True,False


In [49]:
#Check N values to make sure Home and Away columns are false
NFL[NFL["Awaygame"]=='N']

,Week,Day,Time,Winner,Awaygame,Loser,PtsW,PtsL,YdsW,TOW,YdsL,TOL,Tie,PointDiff,YardDiff,TurnoverDiff,Home,Away
Date,,,,,,,,,,,,,,,,,,
2024-02-11,SuperBowl,Sun,6:30PM,Kansas City Chiefs,N,San Francisco 49ers,25.0,22.0,455.0,2.0,382.0,2.0,0,3.0,73.0,0.0,False,False
2023-02-12,SuperBowl,Sun,6:30PM,Kansas City Chiefs,N,Philadelphia Eagles,38.0,35.0,340.0,0.0,417.0,1.0,0,3.0,-77.0,-1.0,False,False
2022-02-13,SuperBowl,Sun,6:30PM,Los Angeles Rams,N,Cincinnati Bengals,23.0,20.0,313.0,2.0,305.0,0.0,0,3.0,8.0,2.0,False,False
2021-02-07,SuperBowl,Sun,6:30PM,Tampa Bay Buccaneers,N,Kansas City Chiefs,31.0,9.0,340.0,0.0,350.0,2.0,0,22.0,-10.0,-2.0,False,False
2020-02-02,SuperBowl,Sun,6:30PM,Kansas City Chiefs,N,San Francisco 49ers,31.0,20.0,397.0,2.0,351.0,2.0,0,11.0,46.0,0.0,False,False
2019-02-03,SuperBowl,Sun,6:30PM,New England Patriots,N,Los Angeles Rams,13.0,3.0,407.0,1.0,260.0,1.0,0,10.0,147.0,0.0,False,False
2018-02-04,SuperBowl,Sun,6:30PM,Philadelphia Eagles,N,New England Patriots,41.0,33.0,538.0,1.0,613.0,1.0,0,8.0,-75.0,0.0,False,False


In [50]:
#Looking at dataframe
NFL

,Week,Day,Time,Winner,Awaygame,Loser,PtsW,PtsL,YdsW,TOW,YdsL,TOL,Tie,PointDiff,YardDiff,TurnoverDiff,Home,Away
Date,,,,,,,,,,,,,,,,,,
2024-09-05,1,Thu,8:20PM,Kansas City Chiefs,,Baltimore Ravens,27.0,20.0,353.0,1.0,452.0,1.0,0,7.0,-99.0,0.0,True,False
2024-09-06,1,Fri,8:15PM,Philadelphia Eagles,,Green Bay Packers,34.0,29.0,410.0,3.0,414.0,1.0,0,5.0,-4.0,2.0,True,False
2024-09-08,1,Sun,1:00PM,Pittsburgh Steelers,@,Atlanta Falcons,18.0,10.0,270.0,0.0,226.0,3.0,0,8.0,44.0,-3.0,False,True
2024-09-08,1,Sun,1:00PM,Buffalo Bills,,Arizona Cardinals,34.0,28.0,352.0,1.0,270.0,1.0,0,6.0,82.0,0.0,True,False
2024-09-08,1,Sun,1:00PM,New Orleans Saints,,Carolina Panthers,47.0,10.0,379.0,1.0,193.0,3.0,0,37.0,186.0,-2.0,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-01-14,Division,Sun,1:05PM,Jacksonville Jaguars,@,Pittsburgh Steelers,45.0,42.0,378.0,0.0,545.0,2.0,0,3.0,-167.0,-2.0,False,True
2018-01-14,Division,Sun,4:40PM,Minnesota Vikings,,New Orleans Saints,29.0,24.0,403.0,1.0,358.0,2.0,0,5.0,45.0,-1.0,True,False
2018-01-21,ConfChamp,Sun,3:05PM,New England Patriots,,Jacksonville Jaguars,24.0,20.0,344.0,1.0,374.0,0.0,0,4.0,-30.0,1.0,True,False


In [51]:
#Combine duplicate teams into one
# Define the mapping to combine the Washington team names and the Raiders team names
team_name_mapping = {
    'Washington Redskins': 'Washington Commanders',
    'Washington Football Team': 'Washington Commanders',
    'Washington Commanders': 'Washington Commanders',
    'Oakland Raiders': 'Las Vegas Raiders',
    'Las Vegas Raiders': 'Las Vegas Raiders'
}

# Replace the team names in both 'Winner' and 'Loser' columns
NFL['Winner'] = NFL['Winner'].replace(team_name_mapping)
NFL['Loser'] = NFL['Loser'].replace(team_name_mapping)

# Check that the replacement was successful
NFL[['Winner', 'Loser']].drop_duplicates().head()

,Winner,Loser
Date,,
2024-09-05,Kansas City Chiefs,Baltimore Ravens
2024-09-06,Philadelphia Eagles,Green Bay Packers
2024-09-08,Pittsburgh Steelers,Atlanta Falcons
2024-09-08,Buffalo Bills,Arizona Cardinals
2024-09-08,New Orleans Saints,Carolina Panthers


In [52]:
#Check for duplicate teams
print(NFL["Winner"].unique())

['Kansas City Chiefs' 'Philadelphia Eagles' 'Pittsburgh Steelers'
 'Buffalo Bills' 'New Orleans Saints' 'Chicago Bears'
 'New England Patriots' 'Houston Texans' 'Miami Dolphins'
 'Minnesota Vikings' 'Seattle Seahawks' 'Los Angeles Chargers'
 'Dallas Cowboys' 'Tampa Bay Buccaneers' 'Detroit Lions'
 'San Francisco 49ers' 'Cleveland Browns' 'Green Bay Packers'
 'Washington Commanders' 'New York Jets' 'Las Vegas Raiders'
 'Arizona Cardinals' 'Atlanta Falcons' 'Indianapolis Colts'
 'New York Giants' 'Denver Broncos' 'Carolina Panthers' 'Baltimore Ravens'
 'Los Angeles Rams' 'Cincinnati Bengals' 'Tennessee Titans'
 'Jacksonville Jaguars']


Team Dataframe

In [53]:
# Initialize an empty DataFrame to hold the stats
team_stats = pd.DataFrame(columns=[
    'Total_Games', 'Win_Rate', 'Avg_PtsScored', 'Avg_PtsAllowed', 'Avg_Yards', 'Avg_YardsAllowed', 'Avg_TurnoverDiff', 
    'Avg_PointDiff', 'Avg_YardDiff',
    'Home_PtsScored', 'Away_PtsScored', 'Home_Yards', 'Away_Yards', 'Home_Turnover', 'Away_Turnover'
])

# Iterate through each team
for team in NFL['Winner'].unique():
    # Total games for the team (count all games where the team was either winner or loser)
    total_games = len(NFL[(NFL['Winner'] == team) | (NFL['Loser'] == team)])  # Count both wins and losses

    # Calculate the win rate based on the number of wins and total games
    win_rate = len(NFL[(NFL['Winner'] == team) & (NFL['Tie'] == False)]) / total_games  # Wins divided by total games played

    # Stats where the team was the winner
    win_games = NFL[(NFL['Winner'] == team) & (NFL['Tie'] == False)]
    sum_pts_scored_win = win_games['PtsW'].sum()  # Sum points scored in wins
    sum_pts_allowed_win = win_games['PtsL'].sum()  # Sum points allowed in losses
    sum_yards_win = win_games['YdsW'].sum()  # Sum yards gained in wins
    sum_yards_allowed_win = win_games['YdsL'].sum()  # Sum yards allowed in losses
    sum_turnover_diff_win = win_games['TurnoverDiff'].sum()  # Sum turnover diff in wins
    sum_point_diff_win = win_games['PointDiff'].sum()  # Sum point diff in wins
    sum_yard_diff_win = win_games['YardDiff'].sum()  # Sum yard diff in wins

    # Stats where the team was the loser
    loss_games = NFL[(NFL['Loser'] == team) | (NFL['Winner'] == team) & (NFL['Tie'] == True)]
    sum_pts_scored_loss = loss_games['PtsL'].sum()  # Sum points scored in losses
    sum_pts_allowed_loss = loss_games['PtsW'].sum()  # Sum points allowed in losses
    sum_yards_loss = loss_games['YdsL'].sum()  # Sum yards scored in losses
    sum_yards_allowed_loss = loss_games['YdsW'].sum()  # Sum yards allowed in losses
    sum_turnover_diff_loss = (-loss_games['TurnoverDiff'].sum())  # Sum turnover diff in losses
    sum_point_diff_loss = (-loss_games['PointDiff'].sum())  # Sum point diff in losses
    sum_yard_diff_loss = (-loss_games['YardDiff'].sum())  # sum yard diff in losses

    #Games where team was Home and Away
    home_games_team = NFL[((NFL['Home'] == True) & (NFL['Winner'] == team)) | ((NFL['Home'] == False) & (NFL['Loser'] == team))]
    away_games_team = NFL[((NFL['Away'] == True) & (NFL['Winner'] == team)) | ((NFL['Away'] == False) & (NFL['Loser'] == team))]

    # Home and Away stats for the team
    home_pts_scored = (
    pd.concat([
        home_games_team[home_games_team['Winner'] == team]['PtsW'],
        home_games_team[home_games_team['Loser'] == team]['PtsL']
    ]).mean() if len(home_games_team) > 0 else 0)
    away_pts_scored = (
    pd.concat([
        away_games_team[away_games_team['Winner'] == team]['PtsW'],
        away_games_team[away_games_team['Loser'] == team]['PtsL']
    ]).mean() if len(away_games_team) > 0 else 0)
    home_yards = (
    pd.concat([
        home_games_team[home_games_team['Winner'] == team]['YdsW'],
        home_games_team[home_games_team['Loser'] == team]['YdsL']
    ]).mean() if len(home_games_team) > 0 else 0)
    away_yards = (
    pd.concat([
        away_games_team[away_games_team['Winner'] == team]['YdsW'],
        away_games_team[away_games_team['Loser'] == team]['YdsL']
    ]).mean() if len(home_games_team) > 0 else 0)
    home_turnover =  (
    pd.concat([
        home_games_team[home_games_team['Winner'] == team]['TOW'],
        home_games_team[home_games_team['Loser'] == team]['TOL']
    ]).mean() if len(home_games_team) > 0 else 0)
    away_turnover =  (
    pd.concat([
        away_games_team[away_games_team['Winner'] == team]['TOW'],
        away_games_team[away_games_team['Loser'] == team]['TOL']
    ]).mean() if len(home_games_team) > 0 else 0)
    
    # Add all stats to the new dataframe
    team_stats.loc[team] = [
        total_games, win_rate, 
        (sum_pts_scored_win + sum_pts_scored_loss) / total_games,  # Average points scored across wins and losses
        (sum_pts_allowed_win + sum_pts_allowed_loss) / total_games,  # Average points allowed across wins and losses
        (sum_yards_win + sum_yards_loss) / total_games,  # Average yards across wins and losses
        (sum_yards_allowed_win + sum_yards_allowed_loss) / total_games,  # Average yards allowed across wins and losses
        (sum_turnover_diff_win + sum_turnover_diff_loss) / total_games,  # Average turnover diff across wins and losses
        (sum_point_diff_win + sum_point_diff_loss) / total_games,  # Average point diff across wins and losses
        (sum_yard_diff_win + sum_yard_diff_loss) / total_games,  # Average yard diff across wins and losses
        
        home_pts_scored, away_pts_scored, home_yards, away_yards, home_turnover, away_turnover
    ]

# Show the new team stats dataframe
team_stats.head()



,Total_Games,Win_Rate,Avg_PtsScored,Avg_PtsAllowed,Avg_Yards,Avg_YardsAllowed,Avg_TurnoverDiff,Avg_PointDiff,Avg_YardDiff,Home_PtsScored,Away_PtsScored,Home_Yards,Away_Yards,Home_Turnover,Away_Turnover
Kansas City Chiefs,154.0,0.759740,27.603896,21.214286,382.824675,346.389610,-0.259740,6.389610,36.435065,26.756098,28.579710,372.731707,399.260870,0.963415,1.260870
Philadelphia Eagles,147.0,0.625850,25.346939,21.408163,358.666667,329.251701,-0.088435,3.938776,29.414966,26.146667,24.774648,362.853333,358.633803,1.093333,1.436620
Pittsburgh Steelers,137.0,0.583942,21.934307,21.109489,331.846715,330.905109,-0.357664,0.824818,0.941606,24.029412,19.869565,341.411765,325.695652,1.264706,1.304348
Buffalo Bills,145.0,0.641379,25.234483,20.296552,356.862069,321.413793,-0.358621,4.937931,35.448276,26.386667,24.000000,353.440000,360.528571,1.146667,1.371429
New Orleans Saints,139.0,0.589928,25.071942,21.021583,351.345324,333.928058,-0.266187,4.050360,17.417266,26.111111,23.955224,359.277778,342.820896,1.236111,0.925373


In [54]:
print(team_stats.describe())
print(team_stats.info())
print(team_stats.shape)

       Total_Games   Win_Rate  Avg_PtsScored  Avg_PtsAllowed   Avg_Yards  \
count     32.00000  32.000000      32.000000       32.000000   32.000000   
mean     138.06250   0.495114      22.742551       22.841715  342.929171   
std        4.88538   0.108503       2.436505        1.618461   20.881935   
min      132.00000   0.305970      17.893939       19.120567  295.272727   
25%      134.00000   0.402239      20.765148       21.835381  330.122648   
50%      137.00000   0.490739      22.733414       22.613322  341.016165   
75%      140.00000   0.585438      24.816278       24.227062  358.746552   
max      154.00000   0.759740      27.603896       25.451128  382.824675   

       Avg_YardsAllowed  Avg_TurnoverDiff  Avg_PointDiff  Avg_YardDiff  \
count         32.000000         32.000000      32.000000     32.000000   
mean         343.796206         -0.000721      -0.099164     -0.867035   
std           13.628909          0.236395       3.622834     27.976912   
min          316.23

In [55]:
team_stats

,Total_Games,Win_Rate,Avg_PtsScored,Avg_PtsAllowed,Avg_Yards,Avg_YardsAllowed,Avg_TurnoverDiff,Avg_PointDiff,Avg_YardDiff,Home_PtsScored,Away_PtsScored,Home_Yards,Away_Yards,Home_Turnover,Away_Turnover
Kansas City Chiefs,154.0,0.759740,27.603896,21.214286,382.824675,346.389610,-0.259740,6.389610,36.435065,26.756098,28.579710,372.731707,399.260870,0.963415,1.260870
Philadelphia Eagles,147.0,0.625850,25.346939,21.408163,358.666667,329.251701,-0.088435,3.938776,29.414966,26.146667,24.774648,362.853333,358.633803,1.093333,1.436620
Pittsburgh Steelers,137.0,0.583942,21.934307,21.109489,331.846715,330.905109,-0.357664,0.824818,0.941606,24.029412,19.869565,341.411765,325.695652,1.264706,1.304348
Buffalo Bills,145.0,0.641379,25.234483,20.296552,356.862069,321.413793,-0.358621,4.937931,35.448276,26.386667,24.000000,353.440000,360.528571,1.146667,1.371429
New Orleans Saints,139.0,0.589928,25.071942,21.021583,351.345324,333.928058,-0.266187,4.050360,17.417266,26.111111,23.955224,359.277778,342.820896,1.236111,0.925373
Chicago Bears,134.0,0.402985,19.910448,21.843284,309.850746,332.820896,-0.052239,-1.932836,-22.970149,20.940299,18.880597,315.626866,304.074627,1.164179,1.552239
New England Patriots,140.0,0.528571,22.885714,20.257143,342.528571,330.057143,-0.200000,2.628571,12.471429,23.929577,22.101449,348.830986,339.028986,1.197183,1.246377
Houston Texans,139.0,0.424460,21.446043,24.258993,332.345324,360.381295,-0.107914,-2.812950,-28.035971,22.742857,20.130435,340.971429,320.434783,1.171429,1.318841
Miami Dolphins,134.0,0.485075,21.843284,24.216418,329.925373,350.350746,0.141791,-2.373134,-20.425373,23.742424,20.000000,340.409091,319.750000,1.318182,1.573529
Minnesota Vikings,138.0,0.594203,23.978261,22.355072,355.268116,347.253623,-0.195652,1.623188,8.014493,24.970588,23.014286,365.014706,347.642857,1.250000,1.314286


In [56]:
# Rounding the columns based on the suggested decimal places for 'team_stats' DataFrame

team_stats['Win_Rate'] = team_stats['Win_Rate'].round(2)           # Round to 2 decimal places
team_stats['Avg_PtsScored'] = team_stats['Avg_PtsScored'].round(2) # Round to 2 decimal places
team_stats['Avg_PtsAllowed'] = team_stats['Avg_PtsAllowed'].round(2) # Round to 2 decimal places
team_stats['Avg_Yards'] = team_stats['Avg_Yards'].round(2)         # Round to 2 decimal places
team_stats['Avg_YardsAllowed'] = team_stats['Avg_YardsAllowed'].round(2) # Round to 2 decimal places
team_stats['Avg_TurnoverDiff'] = team_stats['Avg_TurnoverDiff'].round(2) # Round to 2 decimal places
team_stats['Avg_PointDiff'] = team_stats['Avg_PointDiff'].round(2) # Round to 2 decimal places
team_stats['Avg_YardDiff'] = team_stats['Avg_YardDiff'].round(2)   # Round to 2 decimal places
team_stats['Home_PtsScored'] = team_stats['Home_PtsScored'].round(2) # Round to 2 decimal places
team_stats['Away_PtsScored'] = team_stats['Away_PtsScored'].round(2) # Round to 2 decimal places
team_stats['Home_Yards'] = team_stats['Home_Yards'].round(2)       # Round to 2 decimal places
team_stats['Away_Yards'] = team_stats['Away_Yards'].round(2)       # Round to 2 decimal places
team_stats['Home_Turnover'] = team_stats['Home_Turnover'].round(2) # Round to 2 decimal places
team_stats['Away_Turnover'] = team_stats['Away_Turnover'].round(2) # Round to 2 decimal places# Round to 2 decimal places


In [57]:
print(team_stats)

                       Total_Games  Win_Rate  Avg_PtsScored  Avg_PtsAllowed  \
Kansas City Chiefs           154.0      0.76          27.60           21.21   
Philadelphia Eagles          147.0      0.63          25.35           21.41   
Pittsburgh Steelers          137.0      0.58          21.93           21.11   
Buffalo Bills                145.0      0.64          25.23           20.30   
New Orleans Saints           139.0      0.59          25.07           21.02   
Chicago Bears                134.0      0.40          19.91           21.84   
New England Patriots         140.0      0.53          22.89           20.26   
Houston Texans               139.0      0.42          21.45           24.26   
Miami Dolphins               134.0      0.49          21.84           24.22   
Minnesota Vikings            138.0      0.59          23.98           22.36   
Seattle Seahawks             137.0      0.57          24.12           22.75   
Los Angeles Chargers         136.0      0.51        

In [58]:
#Save Data as CSV
team_stats.to_csv('Team_Stats.csv', index=True)